## Deploy the trigger on the Production Centre

This Jupyter Notebook creates a Web Processing Service request on the Ellip Platform Deployer to deploy the trigger and its associated coordinators on the Production Centre. 

* Set the deployer

In [2]:
deployer = 'https://ec-ecopotential-triggers-deployer.terradue.com'

* Define the trigger information

In [3]:
trigger = dict([('artifact_id', 'tg-ecopotential-00003'),
            ('version', '1.13.2'),
            ('community', 'ec-ecopotential')])

* First do the imports of the Python libraries required

In [4]:
import jenkins
import time
import lxml.etree as etree
import sys
import requests
import os
import string
import hashlib
import urllib2
import pytz
from datetime import datetime
import time
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService

* Connect to the WPS server and do a GetCapabilities request to list the available process:

In [5]:
wps_url = '%s/zoo-bin/zoo_loader.cgi' % deployer

wps = WebProcessingService(wps_url, verbose=False, skip_caps=True)

wps.getcapabilities()

In [6]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'ec_ecopotential_tg_ecopotential_00004_queue_tg_ecopotential_00004_queue_0_1')
(1, 'coordinator_ec_ecopotential_tg_ecopotential_00003_queue_tg_ecopotential_00003_1_12')
(2, 'ec_ecopotential_tg_ecopotential_00003_queue_tg_ecopotential_00003_1_12')
(3, 'coordinator_ec_ecopotential_tg_ecopotential_00003_via_tg_ecopotential_00003_1_12')
(4, 'ec_ecopotential_tg_ecopotential_00004_via_tg_ecopotential_00004_via_0_1')
(5, 'GetStatus')
(6, 'ec_ecopotential_tg_ecopotential_00003_via_tg_ecopotential_00003_1_12')


* Select the **DeployProcess** process to submit a DescribeProcess request and list the inputs:

In [23]:
process_id = 'DeployProcess'
        
process = wps.describeprocess(process_id)

for input in process.dataInputs:
    print(input.identifier)

applicationPackage


* Build the Python dictionary with the inputs:

In [9]:
# TODO confirm the path to via and queue

docker_hub = 'https://store.terradue.com'

ows_context_url = '%s/api/%s/_applications/%s/%s/%s/%s-%s-context.xml' % (docker_hub,
                                                                          trigger['community'], 
                                                                   trigger['community'],  
                                                                   trigger['artifact_id'], 
                                                                   trigger['version'],  
                                                                   trigger['artifact_id'], 
                                                                   trigger['version'])
    

In [8]:
ows_context_url

'https://store.terradue.com/api/ec-ecopotential/_applications/ec-ecopotential/tg-ecopotential-00003/1.13.2/tg-ecopotential-00003-1.13.2-context.xml'

In [26]:
inputs = [('applicationPackage', ows_context_url)]

* Submit the Execute WPS request:

In [19]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(process_id, 
                                           inputs, 
                                           output = [('deployResult', False)])
        
execution_response = execution.submitRequest(etree.tostring(execution_request))
        
execution.parseResponse(execution_response)

* Monitor the request:

In [20]:
monitorExecution(execution)

 owslib.wps.WPSException : {'locator': None, 'code': 't2h5', 'text': '{\n  "errors" : [ {\n    "status" : 401,\n    "message" : "Unauthorized"\n  } ]\n}'}


* Check if the application was successfully deployed

In [21]:
if execution.isSucceded(): 
            
    print 'Application deployed!'
            
else: 

    print 'Application not deployed :-('

Application not deployed :-(
